In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

# create the environment
import janestreet
print('Creating competition environment...', end='')

In [ ]:
env = janestreet.make_env()
print('Finished.')


In [ ]:
train_df = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
features_df = pd.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')
example_test_df = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_test.csv')
example_sample_submission_df = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_sample_submission.csv')

In [ ]:
train = train_df[train_df['weight'] != 0]

In [ ]:
train['action'] = (train['resp'].copy().values > 0).astype(int)

In [ ]:
features = [col for col in list(train_df.columns) if 'feature' in col]
X_train = train.loc[:, features]
y_train = train.loc[:, 'action']

In [ ]:
print('Creating classifier...', end='')
clf = lgb.LGBMClassifier()
print('Finished.')


In [ ]:
print('Training classifier...', end='')
%time clf.fit(X_train, y_train)
print('Finished.')

In [ ]:
print('Creating submissions file...', end='')
rcount = 0
iter_test = env.iter_test()
for (test_df, prediction_df) in iter_test:
    X_test = test_df.loc[:, features]
    y_preds = clf.predict(X_test)
    prediction_df.action = y_preds
    env.predict(prediction_df)
    rcount += len(test_df.index)
print(f'Finished processing {rcount} rows.')
